In [4]:
import tkinter as tk
import numpy as np
import time
from image_conversor import convert_image
import cv2


'''  .+:*=40&@  '''

# Função para formatar a matriz como uma string de matriz
def format_matrix(matrix):
    return '\n'.join([' '.join(row) for row in matrix])

# Captura da câmera
cap = cv2.VideoCapture(0)

length = 5

# Verificar se a câmera foi aberta corretamente
if not cap.isOpened():
    print("Erro ao abrir a câmera.")
    cap.release()
    exit()

# Criar a janela do tkinter
root = tk.Tk()
root.title("Vídeo ASCII")
root.configure(bg="black")  # Definir o background para preto

# Criar um rótulo para exibir as matrizes na janela com fonte menor
label = tk.Label(root, text="", font=("Courier New", 10), bg="black", fg="#777777")
label.pack()

# Função para capturar cada frame da câmera e atualizar o rótulo com a matriz convertida em ASCII
def update_frame():
    while True:
        ret, frame = cap.read()

        if not ret:  # Verifica se a captura falhou ou o vídeo acabou
            break
        frame = convert_image(frame, length)
        formatted_matrix = format_matrix(frame)

        try:
            if root.winfo_exists():  # Verifica se a janela ainda está aberta
                label.config(text=formatted_matrix)
                root.update()
        except tk.TclError:
            break

# Chamar a função para exibir o vídeo da câmera em ASCII
update_frame()

cap.release()  # Liberar a captura da câmera após fechar a janela

root.mainloop()


#
# Transform an image into asc ii .png

In [3]:
from PIL import Image, ImageDraw, ImageFont
import cv2
from image_conversor import convert_image

'''  .:*+=40&@ '''

# Carregar a imagem de exemplo (substitua pelo seu vídeo)
image = cv2.imread('./data/kiss.jpg')

# Função para formatar a matriz como uma string de matriz
def format_matrix(matrix):
    return '\n'.join([' '.join(row) for row in matrix])

# Definir o comprimento da fonte
length = 5

additional_value = round(((-40/3) * length) + (533/3))
print(additional_value)

# Função para extrair o conteúdo da label como uma imagem e salvar em um arquivo PNG
def extract_image(image, formatted_text):
    rows = len(image) 
    cols = len(image[0]) + additional_value
    cell_width, cell_height = font.getsize("X")  # Tamanho exato de cada caractere

    # Adicionar espaço extra entre as linhas
    line_spacing = 1

    width = cols * cell_width - 10 # Largura da imagem
    height = rows * (cell_height + line_spacing)  # Altura da imagem com espaço extra
    image = Image.new("RGB", (width, height), color="black")
    draw = ImageDraw.Draw(image)

    # Desenhar o texto com espaço extra entre as linhas
    y = 0
    for line in formatted_text.split('\n'):
        draw.text((0, y), line, fill="white", font=font)
        y += cell_height + line_spacing

    image.save("output.png")



# Converter o frame para a matriz ASCII
converted_image = convert_image(image, length)
formatted_matrix = format_matrix(converted_image)

# Utilizar a fonte padrão do PIL (fonte monoespaçada)
font = ImageFont.load_default()

# Chamar a função para extrair a imagem automaticamente
extract_image(converted_image, formatted_matrix)


111


/tmp/ipykernel_12705/3236663050.py:22: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  cell_width, cell_height = font.getsize("X")  # Tamanho exato de cada caractere
